# DCGAN  with Keras

##imports




In [0]:
from __future__ import print_function, division

In [2]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

Using TensorFlow backend.


## Making class

In [0]:
class DCGAN():
  def __init__(self):
    #Input shape
    self.img_rows = 28
    self.img_cols = 28
    self.channels = 1
    self.img_shape = (self.img_rows, self.img_cols, self.channels)
    self.latent_dim = 100
    
    optimizer = Adam(0.0002, 0.5)
    
    #build and compile the Discriminator
    self.discriminator = self.build_discriminator()
    self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])
    
    #Build the generator
    self.generator = self.build_generator()
    
    #The generator takes noise as input
    z = Input(shape=(self.latent_dim,))
    img = self.generator(z)
    
    #Only train the generator for the combined model
    self.discriminator.trainable = False
    
    #The discriminator takes generated images as input
    valid = self.discriminator(img)
    
    #The combined model
    self.combined = Model(z, valid)
    self.combined.compile(loss='binary_crossentropy', optimizer = optimizer)
    
    
  def build_generator(self):
      
    model = Sequential()
      
    model.add(Dense(128 * 7 * 7, activation = "relu", input_dim = self.latent_dim))
    model.add(Reshape((7, 7, 128)))
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
    model.add(Activation("tanh"))
      
    model.summary()
      
    noise = Input(shape=(self.latent_dim,))
    img = model(noise)
      
    return Model(noise, img)
    
    
  def build_discriminator(self):
      
    model = Sequential()
      
    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0,1),(0,1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
      
    model.summary()
      
    img = Input(shape=self.img_shape)
    validity = model(img)
      
    return Model(img, validity)
    
  def train(self, epochs, batch_size=128, save_interval=50):
      
    #Load the dataset
    (X_train, _), (_, _) = mnist.load_data()
    
    #Rescale -1 to 1
    X_train = X_train / 127.5 -1.
    X_train = np.expand_dims(X_train, axis=3)
      
    #Adversarial ground truts
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
      
    for epoch in range(epochs):
      #Train Discriminator - Select a random half of images
      idx = np.random.randint(0, X_train.shape[0], batch_size)
      imgs = X_train[idx]
        
      #Sample noise and generate a batch of new images
      noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
      gen_imgs = self.generator.predict(noise)
        
      #Train the discriminator
      d_loss_real = self.discriminator.train_on_batch(imgs, valid)
      
      #print(len(d_loss_real))
      #print(d_loss_real[0].shape)
      #print(d_loss_real[1].shape)
      
      d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
      
      d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
      #Train the Generator
      g_loss = self.combined.train_on_batch(noise, valid)
       

      print ("%d [D loss : %f, acc.: %.2f%%] [G loss : %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
        
      #If at save interval => save generated image samples
      if epoch % save_interval == 0:
        self.save_imgs(epoch)
         
     
  def save_imgs(self, epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, self.latent_dim))
    gen_imgs = self.generator.predict(noise)
      
    #Rescale images 0 ~ 1
    gen_imgs = 0.5 * gen_imgs + 0.5
      
    fig, axs = plt.subplots(r,c)
    cnt = 0
    for i in range(r):
      for j in range(c):
        axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
        axs[i,j].axis('off')
        cnt += 1
    fig.savefig("./mnist_%d.png" % epoch)
    plt.close()

##Run

In [4]:
if __name__ == '__main__':
  dcgan = DCGAN()
  dcgan.train(epochs=4000, batch_size=32, save_interval=50)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 64)          0         
__________

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss : 1.166574, acc.: 35.94%] [G loss : 0.766863]
1 [D loss : 0.746622, acc.: 57.81%] [G loss : 0.844923]
2 [D loss : 0.694246, acc.: 64.06%] [G loss : 0.957335]
3 [D loss : 0.627639, acc.: 67.19%] [G loss : 1.150681]
4 [D loss : 0.573241, acc.: 68.75%] [G loss : 1.164688]
5 [D loss : 0.482080, acc.: 76.56%] [G loss : 1.015404]
6 [D loss : 0.314044, acc.: 92.19%] [G loss : 0.727138]
7 [D loss : 0.333367, acc.: 85.94%] [G loss : 0.596053]
8 [D loss : 0.723773, acc.: 62.50%] [G loss : 1.138627]
9 [D loss : 0.663676, acc.: 65.62%] [G loss : 1.521978]
10 [D loss : 0.592516, acc.: 67.19%] [G loss : 1.637581]
11 [D loss : 0.590128, acc.: 60.94%] [G loss : 1.249079]
12 [D loss : 0.693441, acc.: 62.50%] [G loss : 1.572610]
13 [D loss : 0.462146, acc.: 73.44%] [G loss : 1.573959]
14 [D loss : 0.549460, acc.: 70.31%] [G loss : 1.775704]
15 [D loss : 0.836700, acc.: 57.81%] [G loss : 1.853939]
16 [D loss : 0.682756, acc.: 65.62%] [G loss : 1.465002]
17 [D loss : 0.613160, acc.: 70.31%] [G l